##### データセットの場所やバッチサイズなどの定数値の設定

In [19]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# GPU を使用しない環境（CPU環境）で実行する場合は DEVICE = 'cpu' とする
DEVICE = 'cuda:0'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
# 再開モードの場合も, このエポック数の分だけ追加学習される（N_EPOCHSは最終エポック番号ではない）
N_EPOCHS = 1000

# 学習時のバッチサイズ
BATCH_SIZE = 16

# 訓練データセット（画像ファイルリスト）のファイル名
DATASET_CSV = './tinyCelebA/image_list.csv'

# 画像ファイルの先頭に付加する文字列（データセットが存在するディレクトリのパス）
DATA_DIR = './tinyCelebA/'

# 画像サイズ
H = 128 # 縦幅
W = 128 # 横幅
C = 3 # チャンネル数（カラー画像なら3，グレースケール画像なら1）

# 拡散過程／逆拡散過程（生成過程）のタイムステップ数
N_TIMESTEPS = 1000

# DDIMを用いて, より短時間で生成過程を実行する場合のタイムステップ数
N_GEN_TIMESTEPS = 20

# タイムステップ情報を何次元のベクトルにエンコードするか
TIME_EMBED_DIM = 512

# 学習結果の保存先フォルダ
MODEL_DIR = './DDPM_models/'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, './unet_model.pth')

# 中断／再開の際に用いる一時ファイル
CHECKPOINT_EPOCH = os.path.join(MODEL_DIR, 'checkpoint_epoch.pkl')
CHECKPOINT_MODEL = os.path.join(MODEL_DIR, 'checkpoint_model.pth')
CHECKPOINT_OPT = os.path.join(MODEL_DIR, 'checkpoint_opt.pth')

##### ニューラルネットワークモデルの定義

In [10]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


# タイムステップ情報の埋め込みベクトルを計算する層
#   - time_embed_dim: タイムステップ情報埋め込みベクトルの次元数
class SinusoidalTimeEmbeddings(nn.Module):

    def __init__(self, time_embed_dim):
        super(SinusoidalTimeEmbeddings, self).__init__()
        self.embed_dim = time_embed_dim

    def forward(self, t):
        half_dim = self.embed_dim // 2
        embeddings = torch.log(torch.tensor(10000, device=t.device)) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=t.device) * -embeddings)
        embeddings = t[:, None] * embeddings[None, :]
        embeddings = torch.cat((embeddings.sin(), embeddings.cos()), dim=-1)
        return embeddings


# 先に Group Normalization + Siwsh を適用してから畳み込み処理を実行する畳み込み層
#   - num_groups: Group Nromalization におけるグループ数
class PreNormConv2d(nn.Module):

    def __init__(self, in_channels, out_channels, num_groups, kernel_size, stride, padding, init_scale=1.0):
        super(PreNormConv2d, self).__init__()
        self.act = nn.SiLU()
        self.norm = nn.GroupNorm(num_groups=num_groups, num_channels=in_channels)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=padding, stride=stride)
        nn.init.xavier_uniform_(self.conv.weight, gain=math.sqrt(init_scale or 1e-10))
        nn.init.zeros_(self.conv.bias)

    def forward(self, x):
        h = self.norm(x)
        h = self.act(h)
        return self.conv(h)


# タイムステップ情報を考慮した ResBlock
#   - num_groups: Group Nromalization におけるグループ数
#   - time_embed_dim: タイムステップ情報埋め込みベクトルの次元数（0以下の場合は通常の ResBlock になる）
class DDPMResBlock(nn.Module):

    def __init__(self, in_channels, out_channels, num_groups, kernel_size=3, time_embed_dim=0):
        super(DDPMResBlock, self).__init__()
        if time_embed_dim > 0:
            self.mlp = nn.Sequential(
                nn.SiLU(), 
                nn.Linear(time_embed_dim, out_channels),
            )
        else:
            self.mlp = None
        self.block1 = PreNormConv2d(in_channels, out_channels, num_groups=num_groups, kernel_size=kernel_size, stride=1, padding=kernel_size//2)
        self.block2 = PreNormConv2d(out_channels, out_channels, num_groups=num_groups, kernel_size=kernel_size, stride=1, padding=kernel_size//2, init_scale=0.0)
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x, time_embedding=None):
        h = self.block1(x)
        if self.mlp is not None:
            h = h + self.mlp(time_embedding).unsqueeze(2).unsqueeze(3)
        h = self.block2(h)
        return h + self.skip(x)


# Linear Attention
#   - num_groups: Group Nromalization におけるグループ数
#   - num_heads: マルチヘッドアテンションのヘッド数
#   - embed_dim: 1ヘッドあたりの次元数（タイムステップ情報埋め込みベクトルの次元数とは別）
class DDPMLinearAttention(nn.Module):

    def __init__(self, in_channels, out_channels, num_groups, num_heads, embed_dim):
        super(DDPMLinearAttention, self).__init__()
        self.scale = embed_dim ** (- 0.5)
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.norm = nn.GroupNorm(num_groups=num_groups, num_channels=in_channels)
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.to_qkv = nn.Conv2d(in_channels, num_heads * embed_dim * 3, 1, bias=False)
        self.to_out = nn.Conv2d(num_heads * embed_dim, out_channels, 1)
        nn.init.xavier_uniform_(self.to_out.weight, gain=1e-5)

    # x: 特徴マップ（バッチサイズ, チャンネル数, 縦幅, 横幅の4次元テンソル）
    def forward(self, x):
        B, _, H, W = x.size()
        q, k, v = self.to_qkv(self.norm(x)).chunk(3, dim=1)
        q = torch.reshape(q, (B, self.num_heads, self.embed_dim, H * W))
        k = torch.reshape(k, (B, self.num_heads, self.embed_dim, H * W))
        v = torch.reshape(v, (B, self.num_heads, self.embed_dim, H * W))
        q = q.softmax(dim=-2) * self.scale
        k = k.softmax(dim=-1)
        context = torch.einsum("b h d n, b h e n -> b h d e", k, v).contiguous()
        out = torch.einsum("b h d e, b h d n -> b h e n", context, q).contiguous()
        out = torch.reshape(out, (B, self.num_heads * self.embed_dim, H, W))
        return self.to_out(out) + self.skip(x)


# 通常の Multi-head Attention
#   - num_groups: Group Nromalization におけるグループ数
#   - num_heads: マルチヘッドアテンションのヘッド数
#   - embed_dim: 1ヘッドあたりの次元数（タイムステップ情報埋め込みベクトルの次元数とは別）
class DDPMAttention(nn.Module):

    def __init__(self, in_channels, out_channels, num_groups, num_heads, embed_dim):
        super(DDPMAttention, self).__init__()
        self.scale = embed_dim ** (- 0.5)
        self.num_heads = num_heads
        self.embed_dim = embed_dim
        self.norm = nn.GroupNorm(num_groups=num_groups, num_channels=in_channels)
        self.skip = nn.Identity() if in_channels == out_channels else nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.to_qkv = nn.Conv2d(in_channels, num_heads * embed_dim * 3, 1, bias=False)
        self.to_out = nn.Conv2d(num_heads * embed_dim, out_channels, 1)
        nn.init.xavier_uniform_(self.to_out.weight, gain=1e-5)

    # x: 特徴マップ（バッチサイズ, チャンネル数, 縦幅, 横幅の4次元テンソル）
    def forward(self, x):
        B, _, H, W = x.size()
        q, k, v = self.to_qkv(self.norm(x)).chunk(3, dim=1)
        q = torch.reshape(q, (B, self.num_heads, self.embed_dim, H * W))
        k = torch.reshape(k, (B, self.num_heads, self.embed_dim, H * W))
        v = torch.reshape(v, (B, self.num_heads, self.embed_dim, H * W))
        q = q * self.scale
        sim = torch.einsum("b h d i, b h d j -> b h i j", q, k).contiguous()
        sim = sim - sim.amax(dim=-1, keepdim=True).detach()
        attn = sim.softmax(dim=-1)
        out = torch.einsum("b h i j, b h d j -> b h i d", attn, v).contiguous()
        out = torch.reshape(out.permute(0, 1, 3, 2), (B, self.num_heads * self.embed_dim, H, W))
        return self.to_out(out) + self.skip(x)


# アテンションの種類を選択する関数
def get_attention_block(attention_type, in_channels, out_channels, num_groups, num_heads, embed_dim):
    if attention_type == 'linear':
        attn = DDPMLinearAttention(in_channels=in_channels, out_channels=out_channels, num_groups=num_groups, num_heads=num_heads, embed_dim=embed_dim)
    elif attention_type == 'normal':
        attn = DDPMAttention(in_channels=in_channels, out_channels=out_channels, num_groups=num_groups, num_heads=num_heads, embed_dim=embed_dim)
    elif attention_type == 'none':
        attn = nn.Identity()
    else:
        raise NotImplementedError()
    return attn


# 拡散モデルを実現する U-Net の中間層
#   - time_embed_dim: タイムステップ情報埋め込みベクトルの次元数
#   - num_groups: Group Nromalization におけるグループ数
#   - num_heads: マルチヘッドアテンションのヘッド数（1ヘッドあたりの次元数は channels/num_heads で指定）
#   - attention_type: 'normal'なら通常のマルチヘッドアテンション, 'linear'なら linear attention が使用される. 'none'の場合はアテンションなし
class DDPMMiddleLayer(nn.Module):

    def __init__(self, channels, time_embed_dim, num_groups, num_heads=8, attention_type='none'):
        super(DDPMMiddleLayer, self).__init__()
        embed_dim = channels // num_heads
        self.block1 = DDPMResBlock(in_channels=channels, out_channels=channels, num_groups=num_groups, kernel_size=3, time_embed_dim=time_embed_dim)
        self.block2 = DDPMResBlock(in_channels=channels, out_channels=channels, num_groups=num_groups, kernel_size=3, time_embed_dim=time_embed_dim)
        self.attn = get_attention_block(attention_type, channels, channels, num_groups, num_heads, embed_dim)

    def forward(self, x, time_embedding=None):
        h = self.block1(x, time_embedding)
        h = self.attn(h)
        y = self.block2(h, time_embedding)
        return y


# 拡散モデルを実現する U-Net のダウンサンプリング層
#   - time_embed_dim: タイムステップ情報埋め込みベクトルの次元数
#   - num_groups: Group Nromalization におけるグループ数
#   - num_heads: マルチヘッドアテンションのヘッド数（1ヘッドあたりの次元数は in_channels/num_heads で指定）
#   - attention_type: 'normal'なら通常のマルチヘッドアテンション, 'linear'なら linear attention が使用される. 'none'の場合はアテンションなし
#   - with_downsample: Falseの場合はダウンサンプリングを実行しない
#   - with_skip_output: Falseの場合はスキップ接続用の特徴量を出力しない
class DDPMDownSamplingLayer(nn.Module):

    def __init__(self, in_channels, out_channels, time_embed_dim, num_groups, num_heads=8, attention_type='none', with_downsample=True, with_skip_output=True):
        super(DDPMDownSamplingLayer, self).__init__()
        embed_dim = in_channels // num_heads
        self.with_skip_output = with_skip_output
        self.block1 = DDPMResBlock(in_channels=in_channels, out_channels=in_channels, num_groups=num_groups, kernel_size=3, time_embed_dim=time_embed_dim)
        self.block2 = DDPMResBlock(in_channels=in_channels, out_channels=in_channels, num_groups=num_groups, kernel_size=3, time_embed_dim=time_embed_dim)
        self.attn = get_attention_block(attention_type, in_channels, in_channels, num_groups, num_heads, embed_dim)
        if with_downsample:
            self.down = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=4, stride=2, padding=1)
        else:
            self.down = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x, time_embedding=None):
        h = self.block1(x, time_embedding)
        s = self.attn(h) # このブロックの出力をアップサンプリング層へのスキップ接続として使用
        h = self.block2(s, time_embedding)
        y = self.down(h)
        if self.with_skip_output:
            return s, y
        else:
            return y


# 拡散モデルを実現する U-Net のアップサンプリング層
#   - time_embed_dim: タイムステップ情報埋め込みベクトルの次元数
#   - num_groups: Group Nromalization におけるグループ数
#   - num_heads: マルチヘッドアテンションのヘッド数（1ヘッドあたりの次元数は out_channels/num_heads で指定）
#   - attention_type: 'normal'なら通常のマルチヘッドアテンション, 'linear'なら linear attention が使用される. 'none'の場合はアテンションなし
#   - with_upsample: Falseの場合はダウンサンプリングを実行しない
#   - with_skip_input: Falseの場合はスキップ接続用の特徴量を受け付けない
class DDPMUpSamplingLayer(nn.Module):

    def __init__(self, in_channels, out_channels, time_embed_dim, num_groups, num_heads=8, attention_type='none', with_upsample=True, with_skip_input=True):
        super(DDPMUpSamplingLayer, self).__init__()
        embed_dim = out_channels // num_heads
        block1_out_channels = out_channels * 2 if with_skip_input else out_channels
        self.block1 = DDPMResBlock(in_channels=block1_out_channels, out_channels=out_channels, num_groups=num_groups, kernel_size=3, time_embed_dim=time_embed_dim)
        self.block2 = DDPMResBlock(in_channels=out_channels, out_channels=out_channels, num_groups=num_groups, kernel_size=3, time_embed_dim=time_embed_dim)
        self.attn = get_attention_block(attention_type, out_channels, out_channels, num_groups, num_heads, embed_dim)
        if with_upsample:
            self.up = nn.ConvTranspose2d(in_channels=in_channels, out_channels=out_channels, kernel_size=4, stride=2, padding=1)
        else:
            self.up = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x, s=None, time_embedding=None):
        h = self.up(x)
        if s is not None:
            h = torch.cat((h, s), dim=1)
        h = self.block1(h, time_embedding)
        h = self.attn(h)
        y = self.block2(h, time_embedding)
        return y


# 拡散モデルを実現する U-Net の本体
class DDPM_UNet(nn.Module):

    # C: 入力画像のチャンネル数（グレースケール画像なら1，カラー画像なら3）
    # time_embed_dim: タイムステップ情報をエンコーディングする際のコードベクトルの次元数（偶数）
    # num_groups: Group Nromalization におけるグループ数
    def __init__(self, C, time_embed_dim, num_groups=16):
        super(DDPM_UNet, self).__init__()

        # 層ごとのチャンネル数
        L1_C = 128
        L2_C = 128
        L3_C = 256
        L4_C = 256
        L5_C = 512
        L6_C = 512

        # タイムステップ情報のエンコーディングを担当する層
        self.time_encoder = nn.Sequential(
            SinusoidalTimeEmbeddings(time_embed_dim),
            nn.Linear(time_embed_dim, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )

        # 入力画像に対し最初に適用する畳み込み層
        self.init_conv = nn.Conv2d(in_channels=C, out_channels=L1_C, kernel_size=1, stride=1, padding=0)

        # U-Netのダウンサンプリング層
        self.down1 = DDPMDownSamplingLayer(in_channels=L1_C, out_channels=L2_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.down2 = DDPMDownSamplingLayer(in_channels=L2_C, out_channels=L3_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.down3 = DDPMDownSamplingLayer(in_channels=L3_C, out_channels=L4_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.down4 = DDPMDownSamplingLayer(in_channels=L4_C, out_channels=L5_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.down5 = DDPMDownSamplingLayer(in_channels=L5_C, out_channels=L6_C, time_embed_dim=time_embed_dim, num_groups=num_groups, attention_type='linear')

        # U-Netの中間層
        self.mid = DDPMMiddleLayer(channels=L6_C, time_embed_dim=time_embed_dim, num_groups=num_groups, attention_type='linear')

        # U-Netのアップサンプリング層
        self.up5 = DDPMUpSamplingLayer(in_channels=L6_C, out_channels=L5_C, time_embed_dim=time_embed_dim, num_groups=num_groups, attention_type='linear')
        self.up4 = DDPMUpSamplingLayer(in_channels=L5_C, out_channels=L4_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.up3 = DDPMUpSamplingLayer(in_channels=L4_C, out_channels=L3_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.up2 = DDPMUpSamplingLayer(in_channels=L3_C, out_channels=L2_C, time_embed_dim=time_embed_dim, num_groups=num_groups)
        self.up1 = DDPMUpSamplingLayer(in_channels=L2_C, out_channels=L1_C, time_embed_dim=time_embed_dim, num_groups=num_groups)

        # 最後に実行する畳み込み層
        self.last_conv = PreNormConv2d(in_channels=L1_C, out_channels=C, num_groups=num_groups, kernel_size=1, stride=1, padding=0, init_scale=0.0)

    def forward(self, x, t):
        h = self.init_conv(x) # 最初の畳み込み
        time_embedding = self.time_encoder(t) # タイムステップ情報のエンコーディング
        s1, h = self.down1(h, time_embedding) # ダウンサンプリング層（ s1～s5 はアップサンプリング層へのスキップ接続として使用）
        s2, h = self.down2(h, time_embedding)
        s3, h = self.down3(h, time_embedding)
        s4, h = self.down4(h, time_embedding)
        s5, h = self.down5(h, time_embedding)
        h = self.mid(h, time_embedding) # 中間層
        h = self.up5(h, s5, time_embedding) # アップサンプリング層
        h = self.up4(h, s4, time_embedding)
        h = self.up3(h, s3, time_embedding)
        h = self.up2(h, s2, time_embedding)
        h = self.up1(h, s1, time_embedding)
        y = self.last_conv(h) # 最終畳み込み
        return y

##### ノイズスケジューリングの定義

In [11]:
import torch


# ノイズスケジューラ
class NoiseScheduler:

    def __init__(self, device, method:str='linear', num_timesteps:int=1000, start:float=0.0001, end:float=0.02, s:float=0.008, clip:float=0.999):

        # beta を用意
        if method == 'cosine': # あまり上手く動かない．実装ミス？
            num_timesteps += 1
            T = num_timesteps - 1
            t = torch.arange(0, num_timesteps)
            alpha_bar = torch.cos(0.5 * torch.pi * ((t/T)+s)/(1+s))**2
            alpha_bar = alpha_bar / alpha_bar[0]
            beta = torch.clamp(1.0 - alpha_bar[1:] / alpha_bar[:-1], max=clip)
        elif method == 'quadratic': # 十分なエポック数を試したことがない
            beta = torch.linspace(start**0.5, end**0.5, num_timesteps)**2
        elif method == 'sigmoid': # 一回も試したことがない
            beta = torch.sigmoid(torch.linspace(-6, 6, num_timesteps)) * (end - start) + start
        elif method == 'linear': # 結局これが無難？
            beta = torch.linspace(start, end, num_timesteps)
        else:
            raise NotImplementedError(method)
        self.beta = beta.to(device)

        # alpha, alpha_bar などを用意
        self.alpha = 1.0 - self.beta
        self.alpha_bar = torch.cumprod(self.alpha, axis=0)
        self.alpha_bar_prev = F.pad(self.alpha_bar[:-1], (1, 0), value=1.0)
        self.sqrt_alpha_bar = torch.sqrt(self.alpha_bar)
        self.sqrt_one_minus_alpha_bar = torch.sqrt(1.0 - self.alpha_bar)
        self.sqrt_inv_alpha = torch.sqrt(1.0 / self.alpha)

        # 逆拡散過程実行時に使用する係数を用意
        self.var_coeff = torch.sqrt(self.beta * (1.0 - self.alpha_bar_prev) / (1.0 - self.alpha_bar))
        self.noise_scale_coeff = self.sqrt_inv_alpha * self.beta / self.sqrt_one_minus_alpha_bar

    # タイプステップ t において x0 に正規乱数ノイズを付加したデータを生成
    #   - x0: ノイズ付加前の入力画像（ミニバッチ形式で与える）
    #   - t: タイムステップ（ミニバッチ形式で与える）
    #   - noise: 標準正規分布に従うシードノイズ（Noneの場合は関数内で生成）
    def get_noisy_sample(self, x0, t, noise=None):
        if noise is None:
            noise = torch.randn_like(x0)
        return self.sqrt_alpha_bar[t].reshape(-1, 1, 1, 1) * x0 + self.sqrt_one_minus_alpha_bar[t].reshape(-1, 1, 1, 1) * noise

##### 逆拡散過程（生成過程）を実行する関数の定義

In [12]:
import torch
import numpy as np
from tqdm import tqdm
from mylib.data_io import show_images, to_sigmoid_image


# DDIMによるデータ生成
# こちらは, 学習時よりも少ないタイムステップ数で簡易的に画像を生成したい場合に使用
def DDIM_generate(Z, model, noise_scheduler, n_timesteps, n_gen_timesteps, show_progress=False):

    t_list = np.round(np.linspace(0, n_timesteps-1, n_gen_timesteps)).astype(np.int32)
    s_list = np.concatenate([[0], t_list[:-1]])
    timesteps = np.concatenate([t_list.reshape(-1, 1), s_list.reshape(-1, 1)], axis=1)
    with torch.no_grad():
        for t_idx, s_idx in tqdm(reversed(timesteps), total=n_gen_timesteps):

            # ノイズ推定
            t = t_idx * torch.ones((len(Z),), device=Z.device).long()
            noise = model(Z, t)

            # ノイズ除去
            if t_idx == 0:
                Z = noise_scheduler.sqrt_inv_alpha[t_idx] * Z - noise_scheduler.noise_scale_coeff[t_idx] * noise
            else:
                Z = (noise_scheduler.sqrt_alpha_bar[s_idx] / noise_scheduler.sqrt_alpha_bar[t_idx]) * (Z - noise_scheduler.sqrt_one_minus_alpha_bar[t_idx] * noise)
                Z = Z + noise_scheduler.sqrt_one_minus_alpha_bar[s_idx] * noise

            # 途中経過の保存
            if show_progress:
                Z_cpu = to_sigmoid_image(torch.clamp(Z, min=-1.0, max=1.0)).to('cpu').detach()
                show_images(Z_cpu, num=len(Z), num_per_row=8, title='timestep_{}'.format(t_idx+1), save_fig=False, save_dir=MODEL_DIR)

    return Z


# DDPMによるデータ生成
# こちらの方が拡散モデル本来の逆拡散過程
def DDPM_generate(Z, model, noise_scheduler, n_timesteps, show_progress=False, show_interval=50):

    with torch.no_grad():
        for t_idx in tqdm(reversed(range(0, n_timesteps)), total=n_timesteps):

            # ノイズ推定
            t = t_idx * torch.ones((len(Z),), device=Z.device).long()
            noise = model(Z, t)

            # ノイズ除去
            Z = noise_scheduler.sqrt_inv_alpha[t_idx] * Z - noise_scheduler.noise_scale_coeff[t_idx] * noise
            if t_idx != 0:
                Z = Z + noise_scheduler.var_coeff[t_idx] * torch.randn_like(Z)

            # 途中経過の保存
            if show_progress and (t_idx + 1) % show_interval == 0:
                Z_cpu = to_sigmoid_image(torch.clamp(Z, min=-1.0, max=1.0)).to('cpu').detach()
                show_images(Z_cpu, num=len(Z), num_per_row=8, title='timestep_{}'.format(t_idx+1), save_fig=False, save_dir=MODEL_DIR)

    return Z

##### 訓練データセットの読み込み

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset
from mylib.utility import save_datasets, load_datasets_from_file


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False


# 再開モードの場合は，前回使用したデータセットをロードして使用する
if RESTART_MODE:
    train_dataset, _ = load_datasets_from_file(MODEL_DIR)
    if train_dataset is None:
        print('error: there is no checkpoint previously saved.')
        exit()
    train_size = len(train_dataset)

# そうでない場合は，データセットを読み込む
else:

    # CSVファイルを読み込み, 訓練データセットを用意
    train_dataset = CSVBasedDataset(
        filename = DATASET_CSV,
        items = [
            'File Path' # X
        ],
        dtypes = [
            'image' # Xの型
        ],
        dirname = DATA_DIR,
        img_transform=transforms.CenterCrop((H, W)), # 中央128ピクセル分のみを切り出して使用
        img_range=[-1, 1],
    )
    train_size = len(train_dataset)

    # データセット情報をファイルに保存
    save_datasets(MODEL_DIR, train_dataset)

# 訓練データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)

##### 学習処理の実行
- 拡散モデルの学習には一般に数百〜1000エポック程度が必要となります（大抵の場合, 1日以上プログラムを回し続けることになります）. 
- 最初の10〜20エポック程度で損失関数の値は十分に下がったように見えるかもしれませんが, そこから先の僅かな上積みが生成画像の品質に大きく影響します.
- Paperspace Gradient などのクラウド環境で一気に実行するのは困難だと思いますので, 何回かに分けて少しずつ実行することをおすすめします.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import transforms
from mylib.visualizers import LossVisualizer
from mylib.data_io import show_images, to_sigmoid_image, autosaved_model_name
from mylib.utility import save_checkpoint, load_checkpoint


# 前回の試行の続きを行いたい場合は True にする -> 再開モードになる
RESTART_MODE = False

# 何エポックに1回の割合で学習経過を表示するか（モデル保存処理もこれと同じ頻度で実行）
INTERVAL_FOR_SHOWING_PROGRESS = 10


# エポック番号
INIT_EPOCH = 0 # 初期値
LAST_EPOCH = INIT_EPOCH + N_EPOCHS # 最終値

# データ拡張のための画像変換処理
image_transform = transforms.RandomHorizontalFlip(p=0.5) # 確率0.5で左右反転

# ニューラルネットワークの作成
model = DDPM_UNet(C=C, time_embed_dim=TIME_EMBED_DIM).to(DEVICE)

# 最適化アルゴリズムの指定（ここでは Adam を使用）
optimizer = optim.AdamW(model.parameters(), lr=0.00002)
if not RESTART_MODE:
    lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda i: min((i + 1) / 5000, 1.0)) # 学習率のウォームアップに使用

# 再開モードの場合は，前回チェックポイントから情報をロードして学習再開
if RESTART_MODE:
    INIT_EPOCH, LAST_EPOCH, model, optimizer = load_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_MODEL, CHECKPOINT_OPT, N_EPOCHS, model, optimizer)
    print('')

# 損失関数
loss_func = nn.MSELoss()

# 検証の際に使用する乱数を用意
Z_valid = torch.randn((BATCH_SIZE, C, H, W)).to(DEVICE)

# 損失関数値の可視化器を準備
loss_viz = LossVisualizer(['train loss'], init_epoch=INIT_EPOCH, log_mode=True)

# ノイズスケジューラを準備
noise_scheduler = NoiseScheduler(device=DEVICE, method='linear', num_timesteps=N_TIMESTEPS)


# 勾配降下法による繰り返し学習
for epoch in range(INIT_EPOCH, LAST_EPOCH):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X in tqdm(train_dataloader):
        for param in model.parameters():
            param.grad = None
        X = image_transform(X) # データ拡張
        X0 = X.to(DEVICE)
        t = torch.randint(0, N_TIMESTEPS, (len(X0),), device=DEVICE).long() # タイムステップ情報をバッチごとにランダムに設定
        noise = torch.randn_like(X0) # 正規乱数に従うノイズを用意
        Xt = noise_scheduler.get_noisy_sample(X0, t, noise) # 用意したノイズを付加
        noise_estimated = model(Xt, t) # U-Netを用いてノイズを推定
        loss = loss_func(noise_estimated, noise) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        if not RESTART_MODE:
            lr_scheduler.step() # 学習率のウォームアップ（エポックごとに学習率を変更）
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 可視化器に損失関数の値を登録
    print('train loss = {0:.6f}'.format(avg_loss))
    print('')

    # 検証（学習経過の表示，モデル自動保存）
    if epoch == 0 or (epoch + 1) % INTERVAL_FOR_SHOWING_PROGRESS == 0:
        model.eval()
        if epoch == 0:
            X0 = to_sigmoid_image(X0) # to_sigmoid_image 関数を用い，画素値が 0〜1 の範囲となるように調整する
            show_images(X0.to('cpu').detach(), num=len(X0), num_per_row=8, title='real images', save_fig=False, save_dir=MODEL_DIR) # Real画像の例を表示（最初のエポックのみ）
        with torch.inference_mode():
            Z = DDIM_generate(Z_valid, model, noise_scheduler, n_timesteps=N_TIMESTEPS, n_gen_timesteps=50)
            #Z = DDIM_generate(torch.randn((BATCH_SIZE, C, H, W)).to(DEVICE), model, noise_scheduler, n_timesteps=N_TIMESTEPS, n_gen_timesteps=50) # エポックごとに異なる乱数を使用する場合はこのようにする
        Z_cpu = to_sigmoid_image(torch.clamp(Z, min=-1.0, max=1.0)).to('cpu').detach() # torch.clamp 関数と to_sigmoid_image 関数を用い，画素値が 0〜1 の範囲となるように調整する
        show_images(Z_cpu, num=len(Z), num_per_row=8, title='epoch_{}'.format(epoch + 1), save_fig=False, save_dir=MODEL_DIR)
        torch.save(model.state_dict(), autosaved_model_name(MODEL_FILE, epoch + 1)) # 学習途中のモデルを保存したい場合はこのようにする

    # 現在の学習状態を一時ファイル（チェックポイント）に保存
    save_checkpoint(CHECKPOINT_EPOCH, CHECKPOINT_MODEL, CHECKPOINT_OPT, epoch+1, model, optimizer)

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### 学習済みニューラルネットワークモデルのロード

In [ ]:
import torch


# 拡散モデルの学習には本当に時間がかかるので, 参考までに, 教員の方で事前学習済みモデルを用意しました.
# デフォルトのニューラルネットワークモデルの下で tinyCelebA を用いて学習したものです.
# これを用いたい場合は, 以下の変数の値を True にしてください
LOAD_PRETRAINED_MODEL = True

# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = DDPM_UNet(C=C, time_embed_dim=TIME_EMBED_DIM).to(DEVICE)
if LOAD_PRETRAINED_MODEL:
    if not os.path.isfile('DDPM_pretrained_model_tinyCelebA.pth'):
        !wget "https://tus.box.com/shared/static/e0lhar7em56t4hvknjjh09g1m8qtqdf6.pth" -O DDPM_pretrained_model_tinyCelebA.pth
    model.load_state_dict(torch.load('DDPM_pretrained_model_tinyCelebA.pth'))
else:
    model.load_state_dict(torch.load(MODEL_FILE)) # 最終モデルをロードする場合
    #model.load_state_dict(torch.load(autosaved_model_name(MODEL_FILE, 500))) # 例えば500エポック目のモデルをロードしたい場合は，このようにする

##### テスト処理（正規分布に従ってランダムサンプリングした乱数から逆拡散過程に従って画像を生成）

In [ ]:
import torch
from mylib.data_io import show_images, to_sigmoid_image


model = model.to(DEVICE)
model.eval()

# 生成する画像の枚数
n_gen = 32

# 標準正規分布 N(0, 1^2) に従って適当に乱数画像を作成
Z = torch.randn((n_gen, C, H, W)).to(DEVICE)

# 生成処理（逆拡散過程）を実行し，その結果を表示
with torch.inference_mode():
    Y = DDIM_generate(Z, model, noise_scheduler, n_timesteps=N_TIMESTEPS, n_gen_timesteps=N_GEN_TIMESTEPS, show_progress=True) # 少ないタイムステップ数で簡易的に生成する場合
    #Y = DDPM_generate(Z, model, noise_scheduler, n_timesteps=N_TIMESTEPS, show_progress=True) # 本来の逆拡散過程で生成する場合
    Y_cpu = to_sigmoid_image(torch.clamp(Y, min=-1.0, max=1.0)).to('cpu').detach()
    show_images(Y_cpu, num=len(Y), num_per_row=8, title='DDPM_sample_generated', save_fig=True)